# **지능형 IoT 응용** (2021년 2학기)
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 06. CNN 기초

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 밑바닥부터 시작하는 딥러닝 (Deep Learning from Scratch), 사이토 고키 지음, 개잎맵시 옮김, 한빛미디어
- 딥러닝 텐서플로 교과서, 서지영 지음, 길벗

---

## 6.1 CNN (Convolutional Neural Network)

- CNN은 이미지 인식과 음성 인식 등 다양한 응용에 사용되는 신경망
- 특히 이미지 인식 분야에서 딥러닝을 활용한 기법은 대부분 CNN을 기초로 함

### CNN 구조

- 이전에 우리가 본 신경망은 완전연결 계층(fully-connected layer)으로 구성된 신경망
 - 완전연결(fully-connected layer) 계층: 신경망의 인접하는 계층의 모든 뉴런 사이에 연결이 존재. Dense layer, Affine layer이라고도 함
 
<img src="./06.CNN_기초/06-1.fclayer.png" width="90%" height="90%">
<center>완전연결 계층으로 이루어진 네트워크의 예</center>

 
- CNN의 경우 새롭게 합성곱 계층(Convolution layer), 풀링 계층(Pooling layer)이 추가됨

- 일반적인 CNN 계층 구조
 - 입력층
 - 합성곱층
 - 풀링층
 - 완전연결층
 - 출력층
 
<img src="./06.CNN_기초/06-2.convlayer.png" width="90%" height="90%"> 
<center>CNN으로 이루어진 네트워크의 예</center> 

- Affine-ReLU 연결이 Conv-ReLU-(Pooling)으로 변경된 것으로 볼 수 있음
- 풀링 계층은 생략되기 도 함
- 출력에 가까운 층에서는 Affine-ReLU 구성을 사용함
- 마지막 출력층에서는 Affine-Softmax 조합을 그대로 사용함

## 6.2 합성곱 계층 (Convolution layer)

### 6.2.1 완전연결 계층의 문제점

- 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있음

- 완전연결 계층의 문제점: **데이터의 형상이 무시된다는 점, 즉 데이터에 존재하는 공간적인 정보가 활용될 수 없음**
 - 입력 데이터가 이미지인 경우 이미지는 세로,가로,채널(색상)으로 구성된 3차원 데이터임
 - 3차원 데이터에는 공간적 정보가 담겨 있음. 예를 들어, 공간적으로 가까운 픽셀은 값이 비슷하거나 거리가 먼 픽셀끼리는 별 연관이 없고, RGB의 각 채널은 서로 밀접하게 관련되어 있는 등의 특성이 있음
 
 - 하지만 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화(flatten) 해줘야 함
 - MNIST 데이터셋을 사용할 예제에서는 형상이 (1, 28, 28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine 계층에 입력했음
 
 - 이렇게 되면 네트워크에서 입력 데이터가 처리될 때 3차원 데이터에 존재하는 공간 정보는 고려되지 않고 학습이 이루어짐
 

### 6.2.2 합성곱 연산

- 합성곱 계층은 입력 데이터의 형상을 유지함
 - 3차원 이미지 데이터를 그대로 입력으로 받고 다음 계층에도 3차원 데이터로 전달함
 - 3차원 데이터에 담긴 공간적 특징 정보가 사용될 수 있어서 데이터를 제대로 이해할 가능성이 높음

- 합성곱 계층은 입력 데이터에서 특징(feature)을 추출하는 역할을 수행
 - 합성곱 계층의 입출력 데이터를 특징 맵(feature map)이라고도 함
 
- 합성곱 연산 (Convolution)
 - 입력 데이터에 필터(filter)를 적용하여 결과를 계산 (필터를 커널(kernel)이라고도 함)
 - 입력 데이터와 필터를 포개 놓고 대응되는 숫자끼리 곱한 후 모두 더함
 - 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용
 
 
<img src="./06.CNN_기초/06-3.conv.png" width="60%" height="60%"> 
<center>2차원 합성곱 연산의 예</center> 

- 예제 데이터와 필터의 형상(shape)
 - 입력 데이터 (4,4)
 - 필터 (3,3)
 - 출력 데이터 (2,2)
 
 
- 완전연결 신경망에서 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 신경망의 '가중치'에 해당
- 그리고 CNN에도 편향이 존재함
 - 편향은 항상 하나(1,1)만 존재하여, 하나의 값을 필터를 적용한 모든 원소에 더하는 것임
- 필터를 적용한 결과에 각 원소마다 편향을 더하면 최종 출력 데이터가 됨
 - 위의 예제에서 편향이 3이라면 출력 데이터는 왼쪽 상단부터 차례로 18, 19, 9, 18이 될 것임

### 6.2.3 패딩 (Padding)

- 합성곱 연산을 수행하기 전 입력 데이터 주변을 특정 값(예컨대 0)으로 채우는 것
- 주로 출력 크기를 조정할 목적으로 사용함
 - 예를 들어 (4,4) 입력 데이터에 (3,3) 필터를 적용하면 출력은 (2,2)가 되어 입력보다 2만큼 줄어듬
 - 이는 합성곱 연산을 여러 번 되풀이하는 심층 신경망에서 문제가 될 수 있음
 - 합성곱 연산을 거칠 때마다 크기가 작아지면서 어느 시점에서는 출력 크기가 1이 되어버려서 더 이상 합성곱 연산을 적용할 수 없게 될 수 있음
 - 이러한 문제를 막기 위해 패딩을 사용
 
<img src="./06.CNN_기초/06-4.padding.png" width="60%" height="60%"> 
<center>패딩을 추가한 예</center>  

- 위 예에서는 폭 1인 패딩을 적용
 - 패딩 값은 0으로 채움
- (4,4)인 입력 데이터에 패딩이 추가되어 (6,6)이 됨
- 이 입력에 (3,3) 크기의 필터를 적용하면 (4,4) 크기의 출력 데이터가 생성
- 패딩 폭을 2, 3 등 원하는 정수로 설정할 수 있음
 - 패딩 폭을 2로 하면 입력 데이터의 크기는 (8,8)이 되고, 3으로 하면 (10,10)이 됨


### 6.2.4 스트라이드 (Stride)

- 스트라이드
 - 필터를 적용하는 위치의 간격
 - 위에서 본 예에서는 모두 스트라이드가 1
 - 스트라이드를 2로 하면 필터를 적용하는 윈도우가 두 칸씩 이동함
 
 
<img src="./06.CNN_기초/06-5.stride.png" width="60%" height="60%"> 
<center>스트라이드 2인 합성곱 연산</center>  

- (7,7) 입력에 (3,3) 필터를 적용하고 스트라이드를 2로 하면 출력은 (3,3)
- 스트라이드를 키우면 출력의 크기는 작아짐
- 패딩을 크게 하면 출력 크기가 커진 것과 반대


### 6.2.5 출력 크기 계산

- 아래와 같이 입력 크기, 필터 크기, 패딩, 스트라이드가 주어졌을 때 출력 크기(OH,OW)를 계산할 수 있음
 - 입력 크기 (H,W)
 - 필터 크기 (FH,FW)
 - 패딩 P
 - 스트라이드 S


- OH = (H + 2P - FH) / S + 1
- OW = (W + 2P - FW) / S + 1


#### 계산 예
- 입력 (4,4), 패딩 1, 스트라이드 1, 필터 (3,3) 
 - OH = (4 + 2 - 3) / 1 + 1 = 4
 - OW = (4 + 2 - 3) / 1 + 1 = 4
 
- 입력 (7,7), 패딩 0, 스트라이드 2, 필터 (3,3)
 - OH = (7 + 0 - 3) / 2 + 1 = 3
 - OW = (7 + 0 - 3) / 2 + 1 = 3
 
- 주의할 점은 (H + 2P - FH) / S, (W + 2P - FW) / S 계산 결과는 정수로 나와야 함

### 6.2.6 3차원 데이터의 합성곱 연산

- 3차원 데이터의 경우 채널이 여러 개 존재
    - 필터도 입력 데이터와 동일하게 여러 개 채널에 대해서 존재
    
- 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고 그 결과를 더해서 하나의 출력을 만들어 냄

- 주의할 점
 - 입력 데이터의 채널 수와 필터의 채널 수가 동일해야 함 (아래 예에서 채널 수는 3)
 - 필터의 크기는 원하는 값으로 설정할 수 있지만 모든 채널의 필터가 같은 크기여야 함
 
 
<img src="https://media.vlpt.us/post-images/dscwinterstudy/4d744a50-4199-11ea-8bd7-d9eb4a57ded4/fig-7-9.png" width="70%" height="70%"> 
<center>3차원 데이터 합성곱 연산의 예</center> 
<center>(이미지 출처: https://velog.io/@dscwinterstudy/2020-01-28-1501-%EC%9E%91%EC%84%B1%EB%90%A8-8zk5xihhpz)</center>   


<img src="https://blog.kakaocdn.net/dn/edHwPa/btqyQLhReJx/dNLWwlxr5Pz84I0NVe1BlK/img.jpg" width="70%" height="70%">
<center>(이미지 출처: https://mindsee-ai.tistory.com/17)</center>




### 6.2.7 블록으로 생각하기

- 위의 예처럼 3차원 데이터를 채널 3개인 필터 하나로 합성곱을 수행하면 출력 데이터는 2차원 데이터가 됨
 - 채널이 1개인 특징 맵
 
- 입력 데이터와 필터를 직육면체 블록이라 생각하면 다음과 같은 형상으로 표현됨
 - 채널 수 C, 높이 H, 너비 W인 데이터의 형상을 (C,H,W)
 - 채널 수 C, 필터 높이 FH, 필터 너비 FW의 경우 (C, FH, FW)
 - 이때 출력 데이터는 (1,OH,OW)의 형상을 가짐
 
<img src="./06.CNN_기초/06-6.block.png" width="60%" height="60%"> 


- 출력 데이터도 여러 개의 채널이 존재하도록 하려면 어떻게 해야 하나?
 - 필터를 여러 개 사용
 - 필터를 FN개 적용하면 출력 맵도 FN개 생성됨
 - FN개의 맵을 모으면 형상이 (FN,OH,OW)인 블록이 완성됨
 
<img src="./06.CNN_기초/06-7.filters.png" width="60%" height="60%">  
 

## 6.3 풀링 계층

- 풀링(Pooling)은 세로,가로 방향의 공간을 줄이는 연산 (다운 샘플링하는 효과)
 - 연산량을 감소시키고 주요한 특성 벡터를 추출하여 학습을 효과적으로 할 수 있게 함

- 일정한 영역(예를 들어 2x2 영역)을 원소 하나로 집약하여 크기를 줄임
 - 최대 풀링 (Max pooling): 영역의 최대값을 취함
 - 평균 풀링 (Average pooling): 영역의 평균값을 취함
 - 최소 풀링 (Min pooling): 영역의 최소값을 취함
 
- CNN에서는 주로 최대 풀링 사용


<img src="./06.CNN_기초/06-8.pooling.png" width="80%" height="80%">  
<center>최대 풀링의 예</center>


- 위 예에서는 2x2 최대 풀링을 스트라이드 2로 처리
 - 2x2는 대상 영역의 크기를 의미
 - 2x2 크기의 영역에서 가장 큰 원소 하나를 취함
 - 스트라이드가 2이므로 2x2 윈도우가 2칸 간격으로 이동함
 - 일반적으로 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정
 
 
#### 풀링 계층의 특징

- 학습해야 할 매개변수가 없음
 - 합성곱 계층의 필터 매개변수처럼 학습할 매개변수가 없음
 - 풀링은 대상 영역에서 최대값, 평균값을 취하는 명확한 연산이므로 학습할 것이 없음


- 채널 수가 변하지 않음
 - 입력 데이터의 채널 수 그대로 출력 데이터로 내보냄
 - 풀링 연산은 채널마다 독립적으로 계산


- 입력의 변화에 영향을 적게 받음 (강건함)
 - 입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않음
 
 
#### 풀링 계층 출력 데이터 크기

- 입력 데이터 (C,H,W)
- 풀링 윈도우 크기 F
- 스트라이드 S
- 출력 데이터 (C,OH,OW)
 - OH = (H - F) / S + 1
 - OW = (W - F) / S + 1